# Finding bigfoot (partie 2)

Ce bloc de code charge toutes les dépendances du projet. Ne le modifiez pas!

In [ ]:
import Pkg; Pkg.activate(pwd())
_code_path = joinpath(dirname(Base.active_project()), "code")
modules = ["pkg", "palettes", "confusionmatrix", "mocks", "crossvalidation", "pipelines", "nbc", "variableselection", "transformations", "bootstrap", "show"]
for m in modules
    include(joinpath(_code_path, "$(m).jl"))
end

## Chargement des données

On charge maintenant les données d'entraînement du modèle:

In [ ]:
JLD2.jldopen("training.jld2", "r") do file
    global X = file["X"]
    global y = file["y"]
end;
presences = findall(y);
absences = findall(.!y);

On peut "entraîner" un classificateur *no-skill* et *coin flip* pour évaluer leur performance:

In [ ]:
Cnoskill = noskill(y)
Ccoinflip = coinflip(y)

Quelle est la performance attendue du classificateur *no-skill*? On peut utiliser différentes fonctiones: `mcc`, `ppv`, `npv`, `trueskill`, `tpr`, `fpr`, `tnr`, `fnr`, `accuracy`, *etc.*. Expérimentez avec la matrice `Cnoskill` et `Ccoinflip` en adaptant le code ci-dessous (et gardez une trace de vos résultats!):

In [ ]:
mcc(Cnoskill)

La valeur de MCC pour *no skill* est proche de zéro -- est-ce que c'est le résultat attendu?

## Stratégie de validation croisée

Vous devrez choisir une méthode de validation croisée. Toutes les méthodes utilisent la même interface (vous n'aurez pas besoin de modifier votre code selon la méthode utilisée, mais ne changez pas le nom de la variable `splits`), mais vous devrez expérimenter avec plusieurs méthodes pour obtenir la meilleure évaluation de la performance du modèle:

In [ ]:
splits = kfold(y, X; k=20); # k-folds, k est le nombre de folds
#splits = montecarlo(y, X; n=50, proportion=0.1); # Monte-Carlo, n est le nombre de sous-ensembles, et on peut choisir la proportion des données de validation
#splits = [holdout(y, X; proportion=0.2)]; # Un seul holdout

## Entraînement d'un modèle avec deux variables

Dans le cadre de ce projet, un modèle se compose d'une transformation (aucune par défaut), d'un classificateur (nous allons utiliser NBC), et d'un seuil:

In [ ]:
# Transformations possibles:
# - ZScore
# - RawData
# - MultivariateTransform{Whitening}
# - MultivariateTransform{PCA}
model = SDM(RawData(), NBC(), Thresholder())

Nous allons utiliser deux variables: température et précipitation.

In [ ]:
variables = [1,12]

Au début, les paramètres du modèle ne sont pas initialisés. On va devoir effectuer une étape d'entraînement:

In [ ]:
train!(model, y, X[variables,:])

Remarquez que par défaut, le modèle va optimiser le seuil. On peut vérifier le seuil avec `model.threshold.cutoff` -- le seuil est automatiquement choisi pour maximiser le MCC.

Dans la pratique, au lieu d'entraîner notre modèle avec toutes les données, on veut effectuer une validation croissée pour mieux comprendre sa performance. La fonction `crossvalidate` renvoie une série de matrices de confusion, pour les données de validation et pour les données d'entraînement:

In [ ]:
Cv, Ct = crossvalidate(model, y, X[variables,:], splits; classify=true)

On peut visualiser le résultat de la validation croisée. Nous allons représenter les résultats de chaque split, en montrant à la fois les données d'entraînement (gris) et de validation (orange), pour vérifier que la performance sur les deux jeux de données est comparable.

In [ ]:
fig = Figure()
ax_roc = Axis(fig[1,1]; aspect=DataAspect())
ax_pr = Axis(fig[1,2]; aspect=DataAspect())

scatter!(ax_roc, fpr.(Cv), tpr.(Cv); color=vibrant[2])
scatter!(ax_roc, fpr.(Ct), tpr.(Ct); color=vibrant[1])

scatter!(ax_pr, tpr.(Cv), ppv.(Cv); color=vibrant[2])
scatter!(ax_pr, tpr.(Ct), ppv.(Ct); color=vibrant[1])

for ax in [ax_roc, ax_pr]
    xlims!(ax, 0., 1.)
    ylims!(ax, 0., 1.)
end

current_figure()

On peut maintenant rapporter le MCC moyen de notre modèle:

In [ ]:
mean(mcc.(Cv))

Prenez du temps pour décrire le comportement du modèle, en utilisant par exemple PPV et NPV -- gardez une note de vos résultats.

## Sélection des variables

Dans cette section, nous allons examiner la sélection des variables. Notez que la sélection des variables fait aussi la validation croisée!

In [ ]:
#variables = [1,12] # Sélection manuelle
variables = constrainedselection(model, y, X, splits, variables, mcc; classify=true)
#variables = forwardselection(model, y, X, splits, mcc; classify=true)
#variables = backwardselection(model, y, X, splits, mcc; classify=true)

On peut maintenant faire la validation croisée du modèle avec les meilleures variables. Adaptez le code des exemples précédents pour mesurer la performance du modèle:

In [ ]:
Cv, Ct = crossvalidate(model, y, X[variables,:], splits; classify=true)

## Entraînement du modèle avec les variables finales

In [ ]:
train!(model, y, X[variables,:])

## Prédictions avec le modèle

In [ ]:
predictors = [SpeciesDistributionToolkit._read_geotiff("layers.tiff", SimpleSDMPredictor; bandnumber=v) for v in variables]

Prédiction du modèle:

In [ ]:
prediction = similar(predictors[1])
Threads.@threads for k in keys(prediction)
    input = [predictor[k] for predictor in predictors]
    prediction[k] = predict(model, input; classify=false)
end

Données des observations (source: *trust me, bro*):

In [ ]:
sightings = map(x -> tuple(reverse(parse.(Float64, x))...), split.(readlines("occurrences.csv"), ','))
filter!(r -> 25 <= r[2] <= 55, sightings);
filter!(r -> -130 <= r[1] <= -70, sightings);

Visualisation des prédictions du modèle, projection EPSG 9822:

In [ ]:
fig = Figure()
ax = GeoMakie.GeoAxis(fig[1,1]; dest = "+proj=aea +lat_1=0.0 +lat_2=55.0", coastlines=true, lonlims=extrema(longitudes(prediction)), latlims=extrema(latitudes(prediction)))
surface!(ax, prediction, colormap=reverse(cgrad(iridescent)); shading=false)
scatter!(ax, sightings, color=:orange, markersize=3)
hidedecorations!(ax)
hidespines!(ax)
current_figure()

Visualisation de l'aire de distribution projetée (*uniquement pour la couverture spatiale des données*):

In [ ]:
fig = Figure()
ax = GeoMakie.GeoAxis(fig[1,1]; dest = "+proj=aea +lat_1=0.0 +lat_2=55.0", coastlines=true, lonlims=extrema(longitudes(prediction)), latlims=extrema(latitudes(prediction)))
surface!(ax, replace(prediction .>= model.threshold.cutoff, false => nothing), colormap=[light[2]]); shading=false
scatter!(ax, sightings, color=:orange, markersize=3)
hidedecorations!(ax)
hidespines!(ax)
current_figure()

## Bagging

Création d'un modèle d'ensemble *homogène* avec 25 sous-échantillons des données, tirés avec remise:

In [ ]:
bags = bootstrap(y, X; n=25)
ensemble = Bagging(model, bags)
train!(ensemble, y, X[variables,:])

On va mesurer l'incertitude avec l'écart interquartile, *i.e.* la différence entre les quartiles 25% et 75%:

In [ ]:
function iqr(x)
    if all(isnan.(x))
        return 0.0
    else
        return first(diff(quantile(filter(!isnan, x), [0.25, 0.75])))
    end
end

Prédiction de l'incertitude dans le modèle d'ensemble:

In [ ]:
uncertainty = similar(predictors[1])
Threads.@threads for k in keys(prediction)
    input = [predictor[k] for predictor in predictors]
    uncertainty[k] = predict(ensemble, input; consensus=iqr, classify=false)
end

In [ ]:
fig = Figure()
ax = GeoMakie.GeoAxis(fig[1,1]; dest = "+proj=aea +lat_1=0.0 +lat_2=55.0", coastlines=true, lonlims=extrema(longitudes(prediction)), latlims=extrema(latitudes(prediction)))
surface!(ax, uncertainty; colormap=cgrad(incandescent), shading=false)
hidedecorations!(ax)
hidespines!(ax)
current_figure()

On peut mesurer l'erreur *out of bag*:

In [ ]:
C = outofbag(ensemble, y, X[variables,:], bags; classify=true)
accuracy(C)

Vérification de l'accord entre les modèles de l'ensemble:

In [ ]:
agreement = zeros(ConfusionMatrix, (length(bags), length(bags)))

for i in eachindex(bags), j in eachindex(bags)
    if j >= i
        agreement[i,j] = ConfusionMatrix(
            predict(ensemble.models[i], X[variables,:]),
            convert(Vector{Bool}, predict(ensemble.models[j], X[variables,:]))
        )
        agreement[j,i] = agreement[i,j]
    end
end

In [ ]:
κ.(agreement)

In [ ]:
heatmap(κ.(agreement), colormap=:Greys)